In [1]:
import numpy as np
import matplotlib.pyplot as plt
import lhapdf
import pandas as pd


NNPDF4_nlo = lhapdf.mkPDF('NNPDF40_nlo_as_01180')
data = pd.read_csv("../E288.csv")
alpha = 1/137

def pdf(pdfset, flavor, x, QQ):
    return pdfset.xfxQ(flavor, x, QQ)

def S(k):
    return np.exp(-k**2)

def fDNNQ(QM, b=0.5):
    return np.exp(-b * QM)

def compute_A(x1, x2, qT, QM):
    f_u_x1 = pdf(NNPDF4_nlo, 2, x1, QM) 
    f_ubar_x2 = pdf(NNPDF4_nlo, -2, x2, QM)
    f_u_x2 = pdf(NNPDF4_nlo, 2, x2, QM)
    f_ubar_x1 = pdf(NNPDF4_nlo, -2, x1, QM)

    Sk_contribution = (1/2)*(np.pi)*(np.exp(-qT*qT/2))

    fDNN_contribution = fDNNQ(QM)

    ux1ubarx2_term = x1*x2*f_u_x1*f_ubar_x2*Sk_contribution
    ubarx1ux2_term = x2*x1*f_u_x2*f_ubar_x1*Sk_contribution
    FUU = (ux1ubarx2_term + ubarx1ux2_term) * fDNN_contribution
    cross_section =  FUU*qT*((4*np.pi*alpha)**2)/(9*QM*QM*QM)
    return cross_section


x1_values = data['xA'].values
x2_values = data['xB'].values
qT_values = data['PT'].values
QM_values = data['QM'].values


A_values = np.array([
    compute_A(x1, x2, qT, QM)
    for x1, x2, qT, QM in zip(x1_values, x2_values, qT_values, QM_values)
])

results_df = pd.DataFrame({
    'x1': x1_values,
    'x2': x2_values,
    'qT': qT_values,
    'QM': QM_values,
    'A': A_values
})

results_df.to_csv("pseudodataE288_BQM.csv", index=False)
print("Computed A values saved to A_for_E288kinematics.csv")

LHAPDF 6.5.4 loading /home/ishara/LHAPDF/LHAPDF-install/share/LHAPDF/NNPDF40_nlo_as_01180/NNPDF40_nlo_as_01180_0000.dat
NNPDF40_nlo_as_01180 PDF set, member #0, version 1; LHAPDF ID = 331700
Computed A values saved to A_for_E288kinematics.csv


/home/ishara/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import os

def create_folders(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created successfully!")
    else:
        print(f"Folder '{folder_name}' already exists!")

models_folder = 'Models_withLHAPDF'
results_folder = 'Results_withLHAPDF'
create_folders(models_folder)
create_folders(results_folder)
alpha = 1/137

data = pd.read_csv("pseudodataE288_BQM.csv")
pseudo_df = data
x1_values = tf.constant(data['x1'].values, dtype=tf.float32)
x2_values = tf.constant(data['x2'].values, dtype=tf.float32)
qT_values = tf.constant(data['qT'].values, dtype=tf.float32)
QM_values = tf.constant(data['QM'].values, dtype=tf.float32)
A_true_values = tf.constant(data['A'].values, dtype=tf.float32)

def DNNQ():
    return models.Sequential([
        layers.Input(shape=(1,)), 
        layers.Dense(100, activation='relu6'),
        layers.Dense(300, activation='relu6'),
        layers.Dense(300, activation='relu6'),
        layers.Dense(250, activation='relu6'),
        layers.Dense(250, activation='relu6'),
        layers.Dense(1, activation='exponential')
    ])

def pdf(pdfset, flavor, x, QQ):
    return pdfset.xfxQ(flavor, x, QQ)

def custom_loss(dnnQ, A_true, x1, x2, qT, QM):
    dnnQinputs = tf.reshape(QM, (-1, 1))
    dnnQvals = dnnQ(dnnQinputs)

    f_u_x1 = tf.constant(pdf(NNPDF4_nlo, 2, x1, QM), dtype=tf.float32)
    f_ubar_x2 = tf.constant(pdf(NNPDF4_nlo, -2, x2, QM), dtype=tf.float32)
    f_u_x2 = tf.constant(pdf(NNPDF4_nlo, 2, x2, QM), dtype=tf.float32)
    f_ubar_x1 = tf.constant(pdf(NNPDF4_nlo, -2, x1, QM), dtype=tf.float32)

    pi = tf.constant(np.pi, dtype=tf.float32)
    Sk_contribution = (1 / 2) * pi * tf.exp(-qT**2 / 2)

    ux1ubarx2_term = x1 * x2 * f_u_x1 * f_ubar_x2 * Sk_contribution
    ubarx1ux2_term = x2 * x1 * f_u_x2 * f_ubar_x1 * Sk_contribution
    FUU = ux1ubarx2_term + ubarx1ux2_term
    cross_section = FUU * qT * ((4 * np.pi * alpha) ** 2) / (9 * QM * QM * QM) * dnnQvals
    temploss = tf.abs(cross_section - A_true)
    loss = tf.reduce_mean(temploss)  # MAE loss
    return loss

num_models = 3
for i in range(1, num_models + 1):
    dnnQ = DNNQ()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    epochs = 5000
    print_epochs = 100
    
    losses = []
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss = custom_loss(dnnQ, A_true_values, x1_values, x2_values, qT_values, QM_values)
        grads = tape.gradient(loss, dnnQ.trainable_variables)
        optimizer.apply_gradients(zip(grads, dnnQ.trainable_variables))
        losses.append(loss.numpy())

        if epoch % print_epochs == 0 or epoch == epochs - 1:
            print(f"Model {i} - Epoch {epoch + 1}/{epochs}, Loss: {loss.numpy():.3e}")
    
    model_path = os.path.join(models_folder, f'DNNQ_model_{i}.h5')
    dnnQ.save(model_path)
    print(f"Model {i} saved successfully at {model_path}!")


2025-02-06 13:42:33.729092: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-06 13:42:33.736002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738870953.744632  286053 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738870953.747261  286053 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-06 13:42:33.756503: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Folder 'Models_withLHAPDF' created successfully!
Folder 'Results_withLHAPDF' created successfully!


I0000 00:00:1738870954.503128  286053 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6498 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5


Model 1 - Epoch 1/5000, Loss: 1.607e-09
Model 1 - Epoch 101/5000, Loss: 1.376e-09
Model 1 - Epoch 201/5000, Loss: 1.119e-09
Model 1 - Epoch 301/5000, Loss: 9.251e-10
Model 1 - Epoch 401/5000, Loss: 7.703e-10
Model 1 - Epoch 501/5000, Loss: 6.409e-10
Model 1 - Epoch 601/5000, Loss: 5.304e-10
Model 1 - Epoch 701/5000, Loss: 4.360e-10
Model 1 - Epoch 801/5000, Loss: 3.591e-10
Model 1 - Epoch 901/5000, Loss: 2.957e-10
Model 1 - Epoch 1001/5000, Loss: 2.454e-10
Model 1 - Epoch 1101/5000, Loss: 2.071e-10
Model 1 - Epoch 1201/5000, Loss: 1.799e-10
Model 1 - Epoch 1301/5000, Loss: 1.596e-10
Model 1 - Epoch 1401/5000, Loss: 1.439e-10
Model 1 - Epoch 1501/5000, Loss: 1.319e-10
Model 1 - Epoch 1601/5000, Loss: 1.236e-10
Model 1 - Epoch 1701/5000, Loss: 1.159e-10
Model 1 - Epoch 1801/5000, Loss: 1.095e-10
Model 1 - Epoch 1901/5000, Loss: 1.060e-10
Model 1 - Epoch 2001/5000, Loss: 1.030e-10
Model 1 - Epoch 2101/5000, Loss: 1.002e-10
Model 1 - Epoch 2201/5000, Loss: 9.772e-11
Model 1 - Epoch 2301/50

Model 1 - Epoch 5000/5000, Loss: 8.509e-11
Model 1 saved successfully at Models_withLHAPDF/DNNQ_model_1.h5!
Model 2 - Epoch 1/5000, Loss: 1.734e-09
Model 2 - Epoch 101/5000, Loss: 1.481e-09
Model 2 - Epoch 201/5000, Loss: 1.182e-09
Model 2 - Epoch 301/5000, Loss: 9.682e-10
Model 2 - Epoch 401/5000, Loss: 8.042e-10
Model 2 - Epoch 501/5000, Loss: 6.724e-10
Model 2 - Epoch 601/5000, Loss: 5.612e-10
Model 2 - Epoch 701/5000, Loss: 4.670e-10
Model 2 - Epoch 801/5000, Loss: 3.869e-10
Model 2 - Epoch 901/5000, Loss: 3.200e-10
Model 2 - Epoch 1001/5000, Loss: 2.659e-10
Model 2 - Epoch 1101/5000, Loss: 2.232e-10
Model 2 - Epoch 1201/5000, Loss: 1.914e-10
Model 2 - Epoch 1301/5000, Loss: 1.664e-10
Model 2 - Epoch 1401/5000, Loss: 1.491e-10
Model 2 - Epoch 1501/5000, Loss: 1.344e-10
Model 2 - Epoch 1601/5000, Loss: 1.255e-10
Model 2 - Epoch 1701/5000, Loss: 1.172e-10
Model 2 - Epoch 1801/5000, Loss: 1.102e-10
Model 2 - Epoch 1901/5000, Loss: 1.061e-10
Model 2 - Epoch 2001/5000, Loss: 1.030e-10
M

Model 2 - Epoch 5000/5000, Loss: 8.495e-11
Model 2 saved successfully at Models_withLHAPDF/DNNQ_model_2.h5!
Model 3 - Epoch 1/5000, Loss: 1.821e-09
Model 3 - Epoch 101/5000, Loss: 1.592e-09
Model 3 - Epoch 201/5000, Loss: 1.306e-09
Model 3 - Epoch 301/5000, Loss: 1.088e-09
Model 3 - Epoch 401/5000, Loss: 9.090e-10
Model 3 - Epoch 501/5000, Loss: 7.542e-10
Model 3 - Epoch 601/5000, Loss: 6.272e-10
Model 3 - Epoch 701/5000, Loss: 5.167e-10
Model 3 - Epoch 801/5000, Loss: 4.231e-10
Model 3 - Epoch 901/5000, Loss: 3.447e-10
Model 3 - Epoch 1001/5000, Loss: 2.830e-10
Model 3 - Epoch 1101/5000, Loss: 2.349e-10
Model 3 - Epoch 1201/5000, Loss: 1.980e-10
Model 3 - Epoch 1301/5000, Loss: 1.711e-10
Model 3 - Epoch 1401/5000, Loss: 1.526e-10
Model 3 - Epoch 1501/5000, Loss: 1.373e-10
Model 3 - Epoch 1601/5000, Loss: 1.271e-10
Model 3 - Epoch 1701/5000, Loss: 1.188e-10
Model 3 - Epoch 1801/5000, Loss: 1.111e-10
Model 3 - Epoch 1901/5000, Loss: 1.060e-10
Model 3 - Epoch 2001/5000, Loss: 1.029e-10
M

Model 3 - Epoch 5000/5000, Loss: 8.413e-11
Model 3 saved successfully at Models_withLHAPDF/DNNQ_model_3.h5!
